In [16]:
import numpy as np

In [17]:
import random
csize = np.random.randint(4,15) # randomly generated length
k = np.random.randint(2,csize) # k length to split sequence later into k spilts to get hamming distance
print(" Chromosome length : " , csize * k) # chromosome length such that it can be divided into csize no of k splits
seq = [0]*2  #stores the parent sequences
for i in range(len(seq)):
    seq[i] = "".join(random.choice("AGCT") for _ in range(csize*k)) #randomly generates chromosomes
print(" Parents :", seq)

 Chromosome length :  35
 Parents : ['CGCTTCGTGTACAAAGTGACTGCAGGGGTCCCCAC', 'CCTCTCCTGGAACCCAGCTCCAAGTACGAACGGCT']


In [18]:
def seqsplit(seq,k):
    seqlist =[]
    for i in range(0, len(seq)):
        if(i%k==0):
            kseq =seq[i:i+k] #splitting seq into csize no of k splits 
            # example AGTCGTCA w/ k=4 will be split as AGTC and GTCA 
            seqlist.append(kseq)
    return seqlist

In [19]:
temp_seq = [0]*len(seq) # temp to store the split reads 
for j in range(len(seq)):
    temp_seq[j] = seqsplit(seq[j],k)
print(temp_seq) #csize no of k splits which we will use ti find hammingdist of 

[['CGCTT', 'CGTGT', 'ACAAA', 'GTGAC', 'TGCAG', 'GGGTC', 'CCCAC'], ['CCTCT', 'CCTGG', 'AACCC', 'AGCTC', 'CAAGT', 'ACGAA', 'CGGCT']]


In [20]:
def hammingDist(str1, str2):
    i = 0
    count = 0 
 
    while(i < len(str1)):
        if(str1[i] != str2[i]): #if characters at the same index of the two strings dont match then distance is incremented
            count += 1
        i += 1
    return count

In [21]:
parent = [] # this appends both the parent's split reads together to find the overall adjacency matrix
for i in temp_seq:
    parent.extend(i)


seq_hdist = np.zeros((len(parent),len(parent))) #this is the adjacency matrix based on the distances
for i in range(len(temp_seq)):
    for j in range(len(temp_seq)):
        seq_hdist[i][j] = hammingDist(parent[i],parent[j])

In [22]:
from mip import Model, xsum, minimize, BINARY
from itertools import product 

def TSP_ILP(G):
    
       
    V1 =  range(len(G))
    n, V = len(G), set(V1)   
    model = Model()
    # binary variables indicating if arc (i,j) is used 
    # on the route or not
    x = [[model.add_var(var_type=BINARY) for j in V] for i in V]
        # objective function: minimize the distance
    # continuous variable to prevent subtours: each city will have a
    # different sequential id in the planned route except the 1st one
    y = [model.add_var() for i in V]
    model.objective = minimize(xsum(G[i][j]*x[i][j]+ G[i][j]*x[j][i] \
                               for i in V for j in V))
    
    # constraint : leave each city only once
    for i in V:
        model += xsum(x[i][j] for j in V - {i}) == 1
    # constraint : enter each city only once
    for i in V:
        model += xsum(x[j][i] for j in V - {i}) == 1

    # subtour elimination
    for (i, j) in product(V - {0}, V - {0}):
        if i != j:
            model += y[i] - (n+1)*x[i][j] >= y[j]-n     
        
    # optimizing
    model.optimize()
    # checking if a solution was found
    if model.num_solutions:
        nc = 0 # cycle starts from vertex 0
        cycle = [nc]
        while True:
            nc = [i for i in V if x[nc][i].x >= 0.99][0]
            cycle.append(nc)
            if nc == 0:
                break

    return ( cycle)

In [23]:
path = TSP_ILP(seq_hdist) #prints the indices of the columns to be swapped 
if (len(path)%2!=0): #if path length is odd we will drop off the last (starting column index)
    path.pop()
print("Column indices to be swapped : ", path)
# if column indices to be swapped are [1,6,2,5] then column 1 and 6 are interchanged and columns 2 and 5 are interchanged

Column indices to be swapped :  [0, 13, 2, 12, 8, 9, 7, 6, 3, 1, 10, 5, 4, 11]


In [24]:
ii = 0
while(ii!=len(path)):
    parent[path[ii]], parent[path[ii+1]] = parent[path[ii+1]], parent[path[ii]] #swap the column indices in the path
    #to create mutation
    ii+=2

In [25]:
child = [0]*2 #splitting the mutated concatenated parent list into two equal parts 
child[0] = parent[0:(len(parent)//2)]
child[1]=parent[(len(parent)//2):len(parent)]
ch = [0]*2 #joins the splits together to form the full chromosomes of the children
ch[0] = ''.join((child[0][n]) for n in range(len(child[0])))
ch[1] = ''.join((child[1][n]) for n in range(len(child[1])))
print("Children : ", ch)

Children :  ['CGGCTGTGACACGAACGTGTCAAGTAGCTCCCTCT', 'CCCACAACCCCCTGGGGGTCTGCAGACAAACGCTT']
